In [ ]:
BUCKET = 'cloud-training-demos-ml'
PROJECT = 'cloud-training-demos'
REGION = 'us-central1'

In [ ]:
import os
os.environ['BUCKET'] = BUCKET
os.environ['PROJECT'] = PROJECT
os.environ['REGION'] = REGION
os.environ['TFVERSION'] = '1.8'

In [ ]:
%bash
gcloud config set project $PROJECT
gcloud config set compute/region $REGION

In [ ]:
%%bash
if ! gsutil ls | grep -q gs://${BUCKET}/food_model/preproc; then
  gsutil mb -l ${REGION} gs://${BUCKET}
  # copy canonical set of preprocessed files if you didn't do previous notebook
  gsutil -m cp -R gs://food_model gs://${BUCKET}
fi

In [ ]:
%bash
gsutil ls gs://${BUCKET}/food_model/preproc

In [ ]:

%bash
echo "bucket=${BUCKET}"
rm -rf food-model
export PYTHONPATH=${PYTHONPATH}:${PWD}/food_model
python -m food_model.food_notfood \
  --bucket=${BUCKET} \
  --output_dir=food_model \
  --job-dir=./tmp \
  #--pattern="00000-of-" --train_examples=1 --eval_steps=1

In [ ]:
%bash
OUTDIR=gs://${BUCKET}/food_model
JOBNAME=food_model_$(date -u +%y%m%d_%H%M%S)
echo $OUTDIR $REGION $JOBNAME
gsutil -m rm -rf $OUTDIR
gcloud ml-engine jobs submit training $JOBNAME \
  --region=$REGION \
  --module-name=food_model.food_notfood \
  --package-path=$(pwd)/food_model \
  --job-dir=$OUTDIR \
  --staging-bucket=gs://$BUCKET \
  --scale-tier=STANDARD_1 \
  --runtime-version=$TFVERSION \
  -- \
  #--bucket=${BUCKET} \
  #--output_dir=${OUTDIR} \
  #--train_examples=200000